In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [2]:
# Checking version of tensorflow
import tensorflow as tf
print(tf.__version__)

2.14.0


In [3]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import  Dropout, Dense, Flatten, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential

from tensorflow.keras import layers, Model


# code


In [4]:
# data = 'D:/pw23_sr/Master Dataset'
data = 'D:/pw23_sr/Augmented Dataset'

## preprocess


In [5]:
# Creating a Data Generator Object
from keras.preprocessing.image import ImageDataGenerator

# Creating a Data Generator Object
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale = 1.0/255)

batches = 128
height = 128
width = 128

In [6]:
train_set = train_datagen.flow_from_directory(
    directory = data,
    target_size = (height, width),
    batch_size = batches,
    class_mode ='categorical',
    subset = 'training',
    shuffle= True
)

Found 42399 images belonging to 673 classes.


In [7]:
print(train_set.image_shape)

(128, 128, 3)


In [8]:
valid_set = train_datagen.flow_from_directory(
    directory = data,
    target_size = (height, width),
    batch_size = batches,
    class_mode ='categorical',
    subset = 'validation',
    shuffle= False
)

Found 10095 images belonging to 673 classes.


## model

In [9]:
base = DenseNet121(weights='imagenet', include_top=False, input_shape = (height, width ,3), classes = 673)

for layer in base.layers[:-8]:
    layer.trainable=False

In [10]:
model = Sequential()

model.add(base)

model.add(GlobalAveragePooling2D())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1024, activation ='relu'))
model.add(Dense(512, activation ='relu'))
model.add(BatchNormalization())

model.add(Dropout(0.5))

model.add(Dense(673, activation='softmax'))


In [11]:
# from tensorflow.keras.optimizers import RMSProp
from tensorflow.keras.optimizers.experimental import RMSprop
import tensorflow_addons as tfa

model.compile(optimizer= RMSprop(learning_rate= 0.001), loss = 'categorical_crossentropy', metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall(), tfa.metrics.F1Score(num_classes=673, average='macro')])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 4, 4, 1024)        7037504   
                                                                 
 global_average_pooling2d (  (None, 1024)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 1024)              4096      
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 1024)              1049600   
                                                                 
 dense_1 (Dense)             (None, 512)               5

c:\Users\Student\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [12]:
logdir = 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logdir)

In [13]:
spe = len(train_set) 
vs = len(valid_set) 

history = model.fit(train_set, validation_data=valid_set, epochs = 5, steps_per_epoch = spe, validation_steps = vs, callbacks = [tensorboard_callback])

Epoch 1/5
332/332 [==============================] - 895s 3s/step - loss: 4.5452 - accuracy: 0.2008 - precision: 0.8721 - recall: 0.0309 - f1_score: 0.1947 - val_loss: 4.1729 - val_accuracy: 0.2378 - val_precision: 0.8312 - val_recall: 0.0259 - val_f1_score: 0.2201
Epoch 2/5
332/332 [==============================] - 916s 3s/step - loss: 2.3877 - accuracy: 0.5237 - precision: 0.9111 - recall: 0.2054 - f1_score: 0.5193 - val_loss: 3.5224 - val_accuracy: 0.3135 - val_precision: 0.8273 - val_recall: 0.0992 - val_f1_score: 0.3070
Epoch 3/5
332/332 [==============================] - 940s 3s/step - loss: 1.6891 - accuracy: 0.6498 - precision: 0.9172 - recall: 0.3858 - f1_score: 0.6489 - val_loss: 3.4707 - val_accuracy: 0.3085 - val_precision: 0.6485 - val_recall: 0.1579 - val_f1_score: 0.3128
Epoch 4/5
332/332 [==============================] - 913s 3s/step - loss: 1.3878 - accuracy: 0.6997 - precision: 0.9183 - recall: 0.4889 - f1_score: 0.6995 - val_loss: 3.1977 - val_accuracy: 0.3594 - va

In [24]:
# saving the model in tensorflow format
model.save('./MyModel_tf',save_format='tf')

INFO:tensorflow:Assets written to: ./MyModel_tf\assets


INFO:tensorflow:Assets written to: ./MyModel_tf\assets


In [25]:
# loading the saved model
loaded_model = tf.keras.models.load_model('./MyModel_tf')

In [26]:
logdir = 'logs_retrain'
tensorboard_callback_retrain = tf.keras.callbacks.TensorBoard(log_dir = logdir)

In [27]:
# retraining the model
loaded_model.fit(train_set, validation_data=valid_set, epochs = 5, steps_per_epoch=len(train_set), validation_steps=len(valid_set), callbacks = [tensorboard_callback_retrain])

Epoch 1/5
332/332 [==============================] - 1195s 4s/step - loss: 1.1539 - accuracy: 0.7538 - precision: 0.9224 - recall: 0.5735 - f1_score: 0.7540 - val_loss: 3.2954 - val_accuracy: 0.3582 - val_precision: 0.6622 - val_recall: 0.2263 - val_f1_score: 0.3612
Epoch 2/5
332/332 [==============================] - 1215s 4s/step - loss: 1.0203 - accuracy: 0.7766 - precision: 0.9240 - recall: 0.6247 - f1_score: 0.7769 - val_loss: 3.1936 - val_accuracy: 0.3804 - val_precision: 0.6660 - val_recall: 0.2514 - val_f1_score: 0.3775
Epoch 3/5
332/332 [==============================] - 1217s 4s/step - loss: 0.9468 - accuracy: 0.7907 - precision: 0.9244 - recall: 0.6528 - f1_score: 0.7910 - val_loss: 3.2295 - val_accuracy: 0.3810 - val_precision: 0.6584 - val_recall: 0.2694 - val_f1_score: 0.3801
Epoch 4/5
332/332 [==============================] - 1223s 4s/step - loss: 0.8888 - accuracy: 0.8027 - precision: 0.9259 - recall: 0.6779 - f1_score: 0.8030 - val_loss: 3.1710 - val_accuracy: 0.3980 

In [28]:
# Save the model as HDF5
model.save('densenet.h5')

c:\Users\Student\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [29]:
import pickle

# Save the history object to a pickle file
history_pickle_path = 'densenet.pkl'
with open(history_pickle_path, 'wb') as file:
    pickle.dump(history.history, file)

In [16]:
# # Load the training history from the pickle file
# history_pickle_path = 'inception_40_72.pkl'

# with open(history_pickle_path, 'rb') as file:
#     history = pickle.load(file)

# # Extract training and validation loss from the history
# train_loss = history['loss']
# val_loss = history['val_loss']

# # Plot training and validation loss with specified colors
# epochs = range(1, len(train_loss) + 1)

# plt.plot(epochs, train_loss, 'b-', label='Training Loss')  # Blue color for training loss
# plt.plot(epochs, val_loss, 'orange', label='Validation Loss')  # Orange color for validation loss
# plt.title('Training and Validation Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()

# plt.show()

In [17]:
# # Load the training history from the pickle file
# history_pickle_path = 'inception_40_72.pkl'

# with open(history_pickle_path, 'rb') as file:
#     history = pickle.load(file)

# # Extract training and validation accuracy from the history
# train_accuracy = history['accuracy']
# val_accuracy = history['val_accuracy']

# # Plot training and validation accuracy
# epochs = range(1, len(train_accuracy) + 1)

# plt.plot(epochs, train_accuracy, 'b-', label='Training Accuracy')  # Blue color for training accuracy
# plt.plot(epochs, val_accuracy, 'orange', label='Validation Accuracy')  # Orange color for validation accuracy
# plt.title('Training and Validation Accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()

# plt.show()

# Random

In [18]:
# test_set = test_datagen.flow_from_directory(
#     target_size = (224, 224),
#     batch_size = 32,
#     rescale = 1./255,
#     class_mode = 'categorical'
# )

In [19]:
# import os
# import matplotlib.pyplot as plt
# from PIL import Image

# master_dataset_dir = data
# folder_name = 'A14a'

# folder_path = os.path.join(master_dataset_dir, folder_name)

# for filename in os.listdir(folder_path):
#     image_path = os.path.join(folder_path, filename)

#     try:
#         image = Image.open(image_path)
#         plt.imshow(image)
#         plt.title("File name: " + filename)
#         plt.show()
#     except Exception as e:
#         print(f"Error occurred while processing {filename}: {e}")

In [20]:

# x = layers.Dense(256, activation = 'gelu')(x)
# x = layers.Dropout(0.4)(x)
# x = layers.Dense(128, activation = 'gelu')(x)
# x = layers.Dropout(0.2)(x)
# x = layers.Dense(64, activation = 'gelu')(x)
# x = layers.Dropout(0.1)(x)

# x = layers.Dense(num_classes,activation = 'softmax')(x)

In [21]:
# !pip install split-folders

In [22]:
# import splitfolders
# input_folder= '/content/gdrive/MyDrive/college/capstone/Master Dataset'
# splitfolders.ratio(input_folder, output= "Dataset",
#                    seed=42, ratio= (.8,.2), group_prefix= None)

In [23]:
# !zip -r /content/Dataset.zip /content/Dataset